# Get started with Metrics Tracking and Monitoring

This notebook demonstrates how to use Evidently to:
* Generate a model performance report and calculate associated metrics.
* Log model metrics to MLFlow.
* Store the model in MLFlow as an artifact.
* Store the model performance report in MLFlow as an artifact.

In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import pandas as pd

from evidently.metric_preset import RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from pathlib import Path
from sklearn import ensemble, model_selection

from config import MLFLOW_TRACKING_URI, DATA_DIR, FILENAME, REPORTS_DIR

/Users/mnrozhkov/dev/mlrepa/com/evidently/evidently-mlflow/.venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mnrozhkov/dev/mlrepa/com/evidently/evidently-mlflow/.venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mnr

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [3]:
# Download original dataset with: python src/pipelines/load_data.py 

raw_data = pd.read_csv(f"../{DATA_DIR}/{FILENAME}")
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Define column mapping

In [4]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

In [5]:
column_mapping = ColumnMapping()

column_mapping.target = 'target'
column_mapping.prediction = 'prediction'
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

## Train a Model

In [6]:
sample_data = raw_data.set_index('dteday').loc['2011-01-01 00:00:00':'2011-01-28 23:00:00'].reset_index()

print(sample_data.shape)

(594, 17)


In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    sample_data[numerical_features + categorical_features],
    sample_data[target],
    test_size=0.3
)

In [8]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
regressor.fit(X_train, y_train) 

model_path = Path('../models/model.joblib')
joblib.dump(regressor, model_path)

['../models/model.joblib']

## Build the model validation report

In [9]:
# Calculate predictions 
preds_train = regressor.predict(X_train)
preds_test = regressor.predict(X_test)

In [10]:
X_train['target'] = y_train
X_train['prediction'] = preds_train

X_test['target'] = y_test
X_test['prediction'] = preds_test

In [11]:
X_train.head()

,temp,atemp,hum,windspeed,mnth,hr,weekday,season,holiday,workingday,target,prediction
326,0.26,0.2879,0.56,0.0896,1,2,0,1,0,0,16,15.66
225,0.16,0.1818,0.93,0.1045,1,18,2,1,0,1,95,103.16
80,0.28,0.2576,0.30,0.2985,1,13,3,1,0,1,71,72.74
399,0.38,0.3939,0.58,0.3881,1,16,3,1,0,1,104,97.98
184,0.12,0.1364,0.50,0.1940,1,23,0,1,0,0,6,9.94


In [12]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(
    reference_data=X_train, 
    current_data=X_test,
    column_mapping=column_mapping)

regression_performance_report

In [13]:
# Extract Model Train Metrics from the report

train_report_metrics = regression_performance_report.as_dict()
me = train_report_metrics['metrics'][0]['result']['current']['mean_error']
mae = train_report_metrics['metrics'][0]['result']['current']["mean_abs_error"]

print(me, mae)

1.9906145251396652 11.100558659217878


## Log Model Training and Validation Metrics

### Set up MLFlow

In [14]:
# Set up MLFlow Client
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()

print(f"Client tracking uri: {client.tracking_uri}")

Client tracking uri: http://localhost:5000


In [15]:
with mlflow.start_run() as run: 

    # Show newly created run metadata info
    print("Experiment id: {}".format(run.info.experiment_id))
    print("Run id: {}".format(run.info.run_id))
    print("Run name: {}".format(run.info.run_name))
    print('MLFlow tracking uri:', mlflow.get_tracking_uri())
    print('MLFlow artifact uri:', mlflow.get_artifact_uri())
    run_id = run.info.run_id

    # Log metrics
    mlflow.log_metric('me', round(me, 3))
    mlflow.log_metric('mae', round(mae, 3))
    
    # Log model 
    mlflow.log_artifact(model_path)

Experiment id: 0
Run id: 2ffa035289c24826bf439da1ad4ad4a6
Run name: amusing-conch-852
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/0/2ffa035289c24826bf439da1ad4ad4a6/artifacts


## Log Model Validation reports to MLFlow 

In [16]:
report_path = f"../{REPORTS_DIR}/train_report.html"
regression_performance_report.save_html(report_path)
    
with mlflow.start_run(run_id=run_id):
    
    # Show the run metadata info
    print("Experiment id: {}".format(run.info.experiment_id))
    print("Run id: {}".format(run.info.run_id))
    print("Run name: {}".format(run.info.run_name))
    
    # Log the regression_performance_report as an artifact
    mlflow.log_artifact(report_path)

Experiment id: 0
Run id: 2ffa035289c24826bf439da1ad4ad4a6
Run name: amusing-conch-852
